First, lets do some feature engineering

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

train_input = pd.read_csv('train.csv')
test_input = pd.read_csv('test.csv')
gender_submission = pd.read_csv('gender_submission.csv')
test_target = gender_submission['Survived']
train_input.head()
target = train_input['Survived']
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

train_input = train_input.drop(['Name', 'Ticket', 'Fare', 'Cabin', 'PassengerId', 'Survived'], axis=1)
train_input.head()

train_input.isnull().sum()

Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Embarked      2
dtype: int64

In [3]:
train_input.shape

(891, 6)

Changing categories to numerical values

In [4]:
def change_gender(x):
    if x == 'male':
        return 0
    else:
        return 1
    
def change_embarked(x):
    if x == 'S':
        return 0
    if x == 'C':
        return 1
    else:
        return 2

    
train_input.Sex = train_input.Sex.apply(change_gender)
train_input.Embarked = train_input.Embarked.apply(change_embarked)

train_input.head()
train_input.Age = train_input.Age.fillna(value=train_input.Age.mean())
train_input.isnull().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Embarked    0
dtype: int64

In [5]:
from sklearn.preprocessing import StandardScaler

scale = StandardScaler()
train_input = pd.DataFrame(scale.fit_transform(train_input))

train_input.head()

,0,1,2,3,4,5
0,0.827377,-0.737695,-0.592481,0.432793,-0.473674,-0.571870
1,-1.566107,1.355574,0.638789,0.432793,-0.473674,0.991124
2,0.827377,1.355574,-0.284663,-0.474545,-0.473674,-0.571870
3,-1.566107,1.355574,0.407926,0.432793,-0.473674,-0.571870
4,0.827377,-0.737695,0.407926,-0.474545,-0.473674,-0.571870


Defining the model

In [6]:
import tensorflow as tf
import keras 
from keras.layers import Dense, Dropout, Input
from keras.models import Sequential

model = Sequential()
model.add(Dense(units = 4, input_shape = (6,), activation = 'relu'))
model.add(Dense(units = 8, activation = 'relu'))
model.add(Dense(units = 16, activation = 'relu'))
model.add(Dense(units = 32, activation = 'relu'))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dense(units = 32, activation = 'relu'))
model.add(Dense(units = 16, activation = 'relu'))
model.add(Dense(units = 8, activation = 'relu'))
model.add(Dense(units = 4, activation = 'relu'))
model.add(Dense(units = 1 , activation = 'sigmoid'))

Compiling the model

In [7]:
model.compile(loss = tf.keras.losses.binary_crossentropy, optimizer = tf.keras.optimizers.Adam(), metrics = ['acc'])
model.fit(train_input, target, batch_size = 16, verbose = 2, epochs = 200)

Epoch 1/200
56/56 - 2s - loss: 0.6739 - acc: 0.6139 - 2s/epoch - 28ms/step
Epoch 2/200
56/56 - 0s - loss: 0.5952 - acc: 0.6521 - 78ms/epoch - 1ms/step
Epoch 3/200
56/56 - 0s - loss: 0.5461 - acc: 0.7699 - 64ms/epoch - 1ms/step
Epoch 4/200
56/56 - 0s - loss: 0.5233 - acc: 0.7666 - 67ms/epoch - 1ms/step
Epoch 5/200
56/56 - 0s - loss: 0.4973 - acc: 0.7767 - 68ms/epoch - 1ms/step
Epoch 6/200
56/56 - 0s - loss: 0.4912 - acc: 0.7823 - 64ms/epoch - 1ms/step
Epoch 7/200
56/56 - 0s - loss: 0.4852 - acc: 0.7868 - 62ms/epoch - 1ms/step
Epoch 8/200
56/56 - 0s - loss: 0.4688 - acc: 0.8013 - 82ms/epoch - 1ms/step
Epoch 9/200
56/56 - 0s - loss: 0.4656 - acc: 0.7924 - 82ms/epoch - 1ms/step
Epoch 10/200
56/56 - 0s - loss: 0.4547 - acc: 0.8058 - 78ms/epoch - 1ms/step
Epoch 11/200
56/56 - 0s - loss: 0.4515 - acc: 0.8114 - 80ms/epoch - 1ms/step
Epoch 12/200
56/56 - 0s - loss: 0.4445 - acc: 0.8070 - 84ms/epoch - 1ms/step
Epoch 13/200
56/56 - 0s - loss: 0.4484 - acc: 0.8047 - 64ms/epoch - 1ms/step
Epoch 14/

Testing with test data

In [8]:
test_input = test_input.drop(['Name', 'Ticket', 'Fare', 'Cabin', 'PassengerId'], axis=1)

In [9]:
test_input.head()
test_input.isnull().sum()

Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Embarked     0
dtype: int64

In [10]:
test_input.Age = test_input.Age.fillna(value=test_input.Age.mean())
test_input.isnull().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Embarked    0
dtype: int64

Cleaning test data

In [11]:
test_input.Sex = test_input.Sex.apply(change_gender)
test_input.Embarked = test_input.Embarked.apply(change_embarked)
test_input.head()

,Pclass,Sex,Age,SibSp,Parch,Embarked
0,3,0,34.5,0,0,2
1,3,1,47.0,1,0,0
2,2,0,62.0,0,0,2
3,3,0,27.0,0,0,0
4,3,1,22.0,1,1,0


Scaling test data

In [12]:
test_input = pd.DataFrame(scale.fit_transform(test_input))
test_input.head()

,0,1,2,3,4,5
0,0.873482,-0.755929,0.334993,-0.499470,-0.400248,2.243165
1,0.873482,1.322876,1.325530,0.616992,-0.400248,-0.677841
2,-0.315819,-0.755929,2.514175,-0.499470,-0.400248,2.243165
3,0.873482,-0.755929,-0.259330,-0.499470,-0.400248,-0.677841
4,0.873482,1.322876,-0.655545,0.616992,0.619896,-0.677841


Visualizing test data

In [13]:
gender_submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


Predicting results

In [18]:
def normalize(x):
    return round(x)

test_input.head()
y_test = model.predict(test_input)

y_test = np.vectorize(normalize)(y_test)



14/14 [==============================] - 0s 1000us/step


Testing accuracy

In [20]:
from sklearn.metrics import accuracy_score, classification_report
accuracy = accuracy_score(y_test, test_target)
report = classification_report(y_test, test_target)

print(f"Accuracy: {accuracy}")

Accuracy: 0.8373205741626795
